In [19]:
!sudo apt-get update
!sudo apt-get install stockfish

Get:1 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease [3632 B]
Get:2 https://dl.yarnpkg.com/debian stable InRelease [17.1 kB]                 
Get:3 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease [3961 B]
Get:4 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main amd64 Packages [352 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]                
Get:6 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main all Packages [2938 B]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]      
Get:8 https://dl.yarnpkg.com/debian stable/main all Packages [11.8 kB]         
Get:9 https://dl.yarnpkg.com/debian stable/main amd64 Packages [11.8 kB]       
Get:10 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable/main amd64 Packages [4557 B]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]       
Get:12 http://security.ubuntu.com/ubunt

In [1]:
!pip install --upgrade pip
!pip install chess zstandard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 75.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 76.2 MB/s eta 0:00:00
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147878 sha256=0ef2634b5f8b654db345fb0c1d8d9c8f38cfb6e49f1e0c32907f1d36812e786d
  Stored in directory: /home/codespace/.cache/pip/wheels/83/1f/4e/8f4300f7dd554eb8de70ddfed96e94d3d030ace10c5b53d447
Successfully built chess
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [chess]32m1/2 [chess]


In [29]:
import re

def reinstate_move_numbers(moves):
    tokens = moves.strip().split()
    result = []
    for i in range(0, len(tokens), 2):
        move_number = i // 2 + 1
        if i + 1 < len(tokens):
            result.append(f"{move_number}. {tokens[i]} {tokens[i+1]}")
        else:
            result.append(f"{move_number}. {tokens[i]}")
    return " ".join(result)

def clean_and_reformat_pgn(pgn_text):
    # Trenne PGN in Partien anhand von Leerzeilen zwischen Metadaten + Zügen
    games = re.split(r'\n\s*\n', pgn_text.strip())

    cleaned_games = []

    for game in games:
        # Trenne Header und Züge
        header_lines = []
        move_lines = []

        for line in game.strip().splitlines():
            if line.startswith('['):
                header_lines.append(line)
            else:
                move_lines.append(line)

        header = "\n".join(header_lines)
        moves_raw = " ".join(move_lines)

        # Entferne Kommentare wie { ... }
        moves = re.sub(r"\{[^}]*\}", "", moves_raw)
        # Entferne schwarze Zugnummern (z. B. 1...)
        moves = re.sub(r"\d+\.\.\.", "", moves)
        # Entferne alle Zugnummern (z. B. 1.)
        moves = re.sub(r"\d+\.", "", moves)
        # Reduziere Leerzeichen
        moves = re.sub(r"\s+", " ", moves).strip()
        # Setze Zugnummern korrekt neu
        moves_numbered = reinstate_move_numbers(moves)

        cleaned_game = f"{header}\n\n{moves_numbered}"
        cleaned_games.append(cleaned_game)

    return "\n\n".join(cleaned_games)

# Datei laden
with open("/workspaces/Chess_Stockfish/lichess_sample_10000 (1).pgn", "r", encoding="utf-8") as file:
    raw_pgn = file.read()

# Bereinigen und umformatieren
cleaned_pgn = clean_and_reformat_pgn(raw_pgn)

# In Datei schreiben
with open("/workspaces/Chess_Stockfish/cleaned.pgn", "w", encoding="utf-8") as out_file:
    out_file.write(cleaned_pgn)



In [9]:
def reduce_blank_lines_in_pgn(input_path, output_path):
    with open(input_path, "r", encoding="utf-8") as infile:
        lines = infile.readlines()

    cleaned_lines = []
    current_game = []
    blank_line_count = 0

    def flush_game(game_lines):
        """Schreibe ein einzelnes Spiel mit genau einer Leerzeile nach dem Header."""
        cleaned = []
        header_ended = False
        blank_line_count = 0
        for line in game_lines:
            stripped = line.strip()
            if stripped.startswith("["):  # Header-Zeile
                cleaned.append(line)
                header_ended = False
                blank_line_count = 0
            elif stripped == "":
                if not header_ended:
                    if blank_line_count == 0:
                        cleaned.append("\n")
                    blank_line_count += 1
                    header_ended = True
            else:
                cleaned.append(line)
                blank_line_count = 0
        cleaned.append("\n")  # <== Spiel-Ende: genau eine Leerzeile nach dem Spiel
        return cleaned

    for line in lines:
        if line.strip() == "" and not current_game:
            # Leere Zeile vor einem Spielanfang -> überspringen
            continue
        current_game.append(line)
        if line.strip().startswith("[Event") and len(current_game) > 1:
            # Neues Spiel beginnt -> vorheriges Spiel verarbeiten
            cleaned_lines.extend(flush_game(current_game[:-1]))
            current_game = [line]

    if current_game:
        cleaned_lines.extend(flush_game(current_game))

    with open(output_path, "w", encoding="utf-8") as outfile:
        outfile.writelines(cleaned_lines)
reduce_blank_lines_in_pgn("/workspaces/Chess_Stockfish/cleaned.pgn", "cleaned_one_line.pgn")

In [ ]:
import chess
import chess.pgn

def get_elo_bucket(elo):
    if elo < 1000:
        return "beginner"
    elif elo < 1400:
        return "intermediate"
    elif elo < 1800:
        return "club_player"
    elif elo < 2200:
        return "advanced"
    else:
        return "expert"

def analyze_game_opening(game, engine, game_nr, depth=15):
    board = game.board()
    data = []

    white_elo = int(game.headers.get("WhiteElo", 0))
    black_elo = int(game.headers.get("BlackElo", 0))

    for ply, move in enumerate(game.mainline_moves()):
        if ply > 9:  # nur die ersten 10 Halbzüge
            break

        san_move = board.san(move)
        info_before = engine.analyse(board, chess.engine.Limit(depth=depth))
        score_before = info_before["score"].white().score(mate_score=10000) if not info_before["score"].is_mate() else 10000

        # Engine sucht besten Zug
        best_move = info_before["pv"][0] if "pv" in info_before else None
        best_board = board.copy()
        if best_move:
            best_board.push(best_move)
            info_best = engine.analyse(best_board, chess.engine.Limit(depth=depth))
            score_best = info_best["score"].white().score(mate_score=10000) if not info_best["score"].is_mate() else 10000
        else:
            score_best = None

        # Spiele tatsächlichen Zug
        board.push(move)
        info_after = engine.analyse(board, chess.engine.Limit(depth=depth))
        score_after = info_after["score"].white().score(mate_score=10000) if not info_after["score"].is_mate() else 10000


        if ply == 0 or ply % 2 == 0:
            player = "white"
            elo_level = get_elo_bucket(white_elo)
            move_quality = score_after - score_best
        else:  
            player = "black"
            elo_level = get_elo_bucket(black_elo)  
            move_quality = score_best - score_after 

        data.append({
            "game_nr": game_nr,
            "ply": ply,
            "move": san_move,
            "move_quality": move_quality,
            "player": player,
            "elo_level": elo_level,
            "phase": "opening"
        })

    return data


Funktion to analyse every game 

In [4]:
import chess
import chess.pgn

def get_elo_bucket(elo):
    if elo < 1000:
        return "beginner"
    elif elo < 1400:
        return "intermediate"
    elif elo < 1800:
        return "club_player"
    elif elo < 2200:
        return "advanced"
    else:
        return "expert"
    
def result_to_label(result, player):
    if result == "1-0":
        return "win" if player == "white" else "loss"
    elif result == "0-1":
        return "win" if player == "black" else "loss"
    elif result == "1/2-1/2":
        return "draw"
    else:
        return "unknown"    

def analyze_game(game, engine, game_nr, depth=15):
    board = game.board()
    data = []

    white_elo = int(game.headers.get("WhiteElo", 0))
    black_elo = int(game.headers.get("BlackElo", 0))
    result = game.headers.get("Result", "*")  #"1-0", "0-1", "1/2-1/2"

    for ply, move in enumerate(game.mainline_moves()):
        san_move = board.san(move)

        # Bewertung vor dem Zug
        info_before = engine.analyse(board, chess.engine.Limit(depth=depth))
        score_before = info_before["score"].white().score(mate_score=10000) if not info_before["score"].is_mate() else 10000

        # Engine-Zug berechnen
        best_move = info_before["pv"][0] if "pv" in info_before else None
        if best_move:
            best_board = board.copy()
            best_board.push(best_move)
            info_best = engine.analyse(best_board, chess.engine.Limit(depth=depth))
            score_best = info_best["score"].white().score(mate_score=10000) if not info_best["score"].is_mate() else 10000
        else:
            score_best = None

        # Tatsächlichen Zug spielen
        board.push(move)
        info_after = engine.analyse(board, chess.engine.Limit(depth=depth))
        score_after = info_after["score"].white().score(mate_score=10000) if not info_after["score"].is_mate() else 10000

        # Bewertung
        if ply % 2 == 0:
            player = "white"
            elo_level = get_elo_bucket(white_elo)
            move_quality = score_after - score_best if score_best is not None else None
        else:
            player = "black"
            elo_level = get_elo_bucket(black_elo)
            move_quality = score_best - score_after if score_best is not None else None

        total_plies = len(list(game.mainline_moves()))
        if ply < total_plies * 0.3:
            phase = 'opening'
        elif ply < total_plies * 0.7:
            phase = 'middlegame'
        else:
            phase = 'endgame'

        data.append({
            "game_nr": game_nr,
            "ply": ply,
            "move": san_move,
            "move_quality": move_quality,
            "player": player,
            "elo_level": elo_level,
            "phase": phase,
            "result": result_to_label(result, player)
        })

    return data


In [5]:
import chess
import chess.engine
import chess.pgn
from io import StringIO

PGN_FILE = "/workspaces/Chess_Stockfish/cleaned_one_line.pgn"
engine = chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish")

MAX_GAMES = 10000
game_count = 0
all_data = []


with open(PGN_FILE) as pgn:
    while game_count < MAX_GAMES:
        game = chess.pgn.read_game(pgn)
        if game is None:
            break
        game_count += 1
        print("Spiel "+str(game_count)+"")
        all_data.extend(analyze_game(game, engine, game_nr=game_count))



Spiel 1
Spiel 2
Spiel 3
Spiel 4


KeyboardInterrupt: 

In [20]:
for entry in all_data:  # Zeigt Einträge
    if entry['phase']== 'endgame':
        print(entry)

{'game_nr': 1, 'ply': 40, 'move': 'Bb2', 'move_quality': -336, 'player': 'white', 'elo_level': 'club_player', 'phase': 'endgame', 'result': 'win'}
{'game_nr': 1, 'ply': 41, 'move': 'b5', 'move_quality': -243, 'player': 'black', 'elo_level': 'club_player', 'phase': 'endgame', 'result': 'loss'}
{'game_nr': 1, 'ply': 42, 'move': 'Bc3', 'move_quality': -22, 'player': 'white', 'elo_level': 'club_player', 'phase': 'endgame', 'result': 'win'}
{'game_nr': 1, 'ply': 43, 'move': 'a6', 'move_quality': 8, 'player': 'black', 'elo_level': 'club_player', 'phase': 'endgame', 'result': 'loss'}
{'game_nr': 1, 'ply': 44, 'move': 'h3', 'move_quality': -21, 'player': 'white', 'elo_level': 'club_player', 'phase': 'endgame', 'result': 'win'}
{'game_nr': 1, 'ply': 45, 'move': 'Nd5', 'move_quality': 14, 'player': 'black', 'elo_level': 'club_player', 'phase': 'endgame', 'result': 'loss'}
{'game_nr': 1, 'ply': 46, 'move': 'Rfe1', 'move_quality': -43, 'player': 'white', 'elo_level': 'club_player', 'phase': 'endga

•Opening Theory Compliance – Do players follow book openings correctly, or do they deviate early?•Strategic Conformance – How well do players adhere to established positional and tactical guidelines?
•Endgame Fundamentals – Are winning endgame techniques executed correctly, or do deviations impact results?•Skill-Based Differences – Do higher-rated players exhibit greater conformance, and how does deviation affect performance

Whole game 

In [18]:
move_qualities = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None]
avg_move_quality = sum(move_qualities) / len(move_qualities)
print(f"Average move_quality whole game all elo levels: {avg_move_quality}")

Average move_quality whole game all elo levels: -12.553398058252426


In [19]:
move_qualities = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['player'] == 'white']
avg_move_quality = sum(move_qualities) / len(move_qualities)
print(f"Average move_quality white whole game all elo levels: {avg_move_quality}")

Average move_quality white whole game all elo levels: 20.553398058252426


In [16]:
number_games_win = len([entry["game_nr"] for entry in all_data if entry['result'] == 'win' and entry["ply"]<2])
print(f"Number of games won all elo levels: {number_games_win}")
percentage_games_won = number_games_win/len([entry["game_nr"] for entry in all_data if entry["ply"]<2])
print(f"Percentage of games won all elo levels: {percentage_games_won}")

Number of games won all elo levels: 3
Percentage of games won all elo levels: 0.5


In [13]:
#Beginner
move_qualities_beginner = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'beginner']
avg_move_quality_beginner = sum(move_qualities_beginner) / len(move_qualities_beginner)
print(f"Average move_quality whole game beginner: {avg_move_quality_beginner}")

# Intermediate
move_qualities_intermediate = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'intermediate']
avg_move_quality_intermediate = sum(move_qualities_intermediate) / len(move_qualities_intermediate)
print(f"Average move_quality whole game intermediate: {avg_move_quality_intermediate}")

# Club Player
move_qualities_club = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'club_player']
avg_move_quality_club = sum(move_qualities_club) / len(move_qualities_club)
print(f"Average move_quality whole game club_player: {avg_move_quality_club}")

# Advanced
move_qualities_advanced = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'advanced']
avg_move_quality_advanced = sum(move_qualities_advanced) / len(move_qualities_advanced)
print(f"Average move_quality whole game advanced: {avg_move_quality_advanced}")

# Expert
move_qualities_expert = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'expert']
avg_move_quality_expert = sum(move_qualities_expert) / len(move_qualities_expert)
print(f"Average move_quality whole game expert: {avg_move_quality_expert}")


ZeroDivisionError: division by zero

Phase Opening

In [ ]:
move_qualities_opening = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry["phase"] == 'opening']
avg_move_quality_opening = sum(move_qualities_opening) / len(move_qualities_opening)
print(f"Average move_quality opening all elo levels: {avg_move_quality_opening}")

Average move_quality opening all elo levels: -32.30379746835443


In [42]:
#Beginner
move_qualities_beginner_opening = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'beginner' and entry["phase"] == 'opening']
avg_move_quality_beginner_opening = sum(move_qualities_beginner_opening) / len(move_qualities_beginner_opening)
print(f"Average move_quality opening beginner: {avg_move_quality_beginner_opening}")

# Intermediate
move_qualities_intermediate_opening = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'intermediate' and entry["phase"] == 'opening']
avg_move_quality_intermediate_opening = sum(move_qualities_intermediate_opening) / len(move_qualities_intermediate_opening)
print(f"Average move_quality opening intermediate: {avg_move_quality_intermediate_opening}")

# Club Player
move_qualities_club_opening = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'club_player' and entry["phase"] == 'opening']
avg_move_quality_club_opening = sum(move_qualities_club_opening) / len(move_qualities_club_opening)
print(f"Average move_quality opening club_player: {avg_move_quality_club_opening}")

# Advanced
move_qualities_advanced_opening = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'advanced' and entry["phase"] == 'opening']
avg_move_quality_advanced_opening = sum(move_qualities_advanced_opening) / len(move_qualities_advanced_opening)
print(f"Average move_quality opening advanced: {avg_move_quality_advanced_opening}")

# Expert
move_qualities_expert_opening = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'expert' and entry["phase"] == 'opening']
avg_move_quality_expert_opening = sum(move_qualities_expert_opening) / len(move_qualities_expert_opening)
print(f"Average move_quality opening expert: {avg_move_quality_expert_opening}")


ZeroDivisionError: division by zero

In [ ]:
#Beginner
move_qualities_beginner_opening = [entry["result"] for entry in all_data if entry["result"] is not None and entry['elo_level'] == 'beginner' and entry["phase"] == 'opening']

avg_move_quality_beginner_opening = count(move_qualities_beginner_opening) / len(move_qualities_beginner_opening)
print(f"Average move_quality opening beginner: {avg_move_quality_beginner_opening}")

# Intermediate
move_qualities_intermediate_opening = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'intermediate' and entry["phase"] == 'opening']
avg_move_quality_intermediate_opening = sum(move_qualities_intermediate_opening) / len(move_qualities_intermediate_opening)
print(f"Average move_quality opening intermediate: {avg_move_quality_intermediate_opening}")

# Club Player
move_qualities_club_opening = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'club_player' and entry["phase"] == 'opening']
avg_move_quality_club_opening = sum(move_qualities_club_opening) / len(move_qualities_club_opening)
print(f"Average move_quality opening club_player: {avg_move_quality_club_opening}")

# Advanced
move_qualities_advanced_opening = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'advanced' and entry["phase"] == 'opening']
avg_move_quality_advanced_opening = sum(move_qualities_advanced_opening) / len(move_qualities_advanced_opening)
print(f"Average move_quality opening advanced: {avg_move_quality_advanced_opening}")

# Expert
move_qualities_expert_opening = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'expert' and entry["phase"] == 'opening']
avg_move_quality_expert_opening = sum(move_qualities_expert_opening) / len(move_qualities_expert_opening)
print(f"Average move_quality opening expert: {avg_move_quality_expert_opening}")


Phase Middlegame

In [40]:
move_qualities_middle = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry["phase"] == 'middlegame']
avg_move_quality_middle = sum(move_qualities_middle) / len(move_qualities_middle)
print(f"Average move_quality middlegame all elo levels: {avg_move_quality_middle}")

Average move_quality middlegame all elo levels: -64.5047619047619


In [43]:
# Beginner
move_qualities_beginner_middlegame = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'beginner' and entry["phase"] == 'middlegame']
avg_move_quality_beginner_middlegame = sum(move_qualities_beginner_middlegame) / len(move_qualities_beginner_middlegame)
print(f"Average move_quality middlegame beginner: {avg_move_quality_beginner_middlegame}")

# Intermediate
move_qualities_intermediate_middlegame = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'intermediate' and entry["phase"] == 'middlegame']
avg_move_quality_intermediate_middlegame = sum(move_qualities_intermediate_middlegame) / len(move_qualities_intermediate_middlegame)
print(f"Average move_quality middlegame intermediate: {avg_move_quality_intermediate_middlegame}")

# Club Player
move_qualities_club_middlegame = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'club_player' and entry["phase"] == 'middlegame']
avg_move_quality_club_middlegame = sum(move_qualities_club_middlegame) / len(move_qualities_club_middlegame)
print(f"Average move_quality middlegame club_player: {avg_move_quality_club_middlegame}")

# Advanced
move_qualities_advanced_middlegame = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'advanced' and entry["phase"] == 'middlegame']
avg_move_quality_advanced_middlegame = sum(move_qualities_advanced_middlegame) / len(move_qualities_advanced_middlegame)
print(f"Average move_quality middlegame advanced: {avg_move_quality_advanced_middlegame}")

# Expert
move_qualities_expert_middlegame = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'expert' and entry["phase"] == 'middlegame']
avg_move_quality_expert_middlegame = sum(move_qualities_expert_middlegame) / len(move_qualities_expert_middlegame)
print(f"Average move_quality middlegame expert: {avg_move_quality_expert_middlegame}")


ZeroDivisionError: division by zero

Endgame

In [45]:
move_qualities_endgame = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry["phase"] == 'endgame']
avg_move_quality_endgame = sum(move_qualities_endgame) / len(move_qualities_endgame)
print(f"Average move_quality endgame all elo levels: {avg_move_quality_endgame}")

Average move_quality endgame all elo levels: 22.0


In [44]:
# Beginner
move_qualities_beginner_endgame = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'beginner' and entry["phase"] == 'endgame']
avg_move_quality_beginner_endgame = sum(move_qualities_beginner_endgame) / len(move_qualities_beginner_endgame)
print(f"Average move_quality endgame beginner: {avg_move_quality_beginner_endgame}")

# Intermediate
move_qualities_intermediate_endgame = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'intermediate' and entry["phase"] == 'endgame']
avg_move_quality_intermediate_endgame = sum(move_qualities_intermediate_endgame) / len(move_qualities_intermediate_endgame)
print(f"Average move_quality endgame intermediate: {avg_move_quality_intermediate_endgame}")

# Club Player
move_qualities_club_endgame = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'club_player' and entry["phase"] == 'endgame']
avg_move_quality_club_endgame = sum(move_qualities_club_endgame) / len(move_qualities_club_endgame)
print(f"Average move_quality endgame club_player: {avg_move_quality_club_endgame}")

# Advanced
move_qualities_advanced_endgame = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'advanced' and entry["phase"] == 'endgame']
avg_move_quality_advanced_endgame = sum(move_qualities_advanced_endgame) / len(move_qualities_advanced_endgame)
print(f"Average move_quality endgame advanced: {avg_move_quality_advanced_endgame}")

# Expert
move_qualities_expert_endgame = [entry["move_quality"] for entry in all_data if entry["move_quality"] is not None and entry['elo_level'] == 'expert' and entry["phase"] == 'endgame']
avg_move_quality_expert_endgame = sum(move_qualities_expert_endgame) / len(move_qualities_expert_endgame)
print(f"Average move_quality endgame expert: {avg_move_quality_expert_endgame}")


ZeroDivisionError: division by zero